In [1]:
#1 initialize population
import random

def initialize_population(count, N):
    population = []
    for _ in range(count):
        perm = random.sample(range(1, N + 1), N)
        population.append(perm)
    return population

In [2]:
def evaluate_fitness(perm, weigths):
    fitness_scores = 0
    for i in range(len(perm)):
        fitness_scores += weights[i][perm[i]-1]
        for j in range(i+1, len(perm)):
            if(perm[i] == perm[j] or abs(i - perm[i] - 1) == abs(j - perm[j] - 1)):
                fitness_scores -= weights[i][perm[i]-1]
        
    return fitness_scores

In [3]:
def tournament_selection(population, fitness_values, tournament_size):
    selected = []
    for _ in range(len(population)):
        participants = random.sample(range(len(population)), tournament_size)
        winner = max(participants, key=lambda x: fitness_values[x])
        selected.append(population[winner])
    return selected

In [4]:
def crossover(parent1, parent2):
    # create two new offspring by combining the parents
    # IN:  parents
    # OUT: offspring
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point]
    child2 = parent2[:point]
    for i in range(len(parent1)):
        if(parent1[i] not in child2):
            child2.append(parent1[i])
        if(parent2[i] not in child1):
            child1.append(parent2[i])
    return child1, child2

In [10]:
def mutation(individual, mutation_rate):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            point = random.randint(1, len(individual) - 1)
            aux = individual[point]
            individual[point] = individual[i]
            individual[i] = aux
    return individual

In [12]:
def selection(population, fitness_values, num_survivors):
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    survivors = random.choices(population, probabilities, k=num_survivors)
    return survivors

In [7]:
def check_constraints(solution, N):
    # one queen in each row and column
    if len(set(solution)) != N:
        return False
    # two queens shouldn't be placed in the same diagonal
    for i in range(N):
        for j in range(i + 1, N):
            if abs(i - j) == abs(solution[i] - solution[j]):
                return False
    return True

In [57]:
def genetic_algorithm(population_size, chromosome_length, generations, mutation_rate, weights):
    # initialize the population
    population = initialize_population(population_size, chromosome_length) 
    best_solution = None
    best_fitness = -1

    for x in range(generations):
        # Fitness evaluation
        fitness_scores = []
        for i in range(len(population)):
            fitness_scores.append(evaluate_fitness(population[i], weights))
        
        # Selection
        selected_population = tournament_selection(population, fitness_scores, 5)
        # Crossover
        offsprings = []
        for i in range(0, len(selected_population), 2):
            parent1 = selected_population[i]
            parent2 = selected_population[i + 1]
            child1, child2 = crossover(parent1, parent2)
            offsprings.extend([child1, child2])

        # Mutation
        mutated_offspring_population = [mutation(individual, mutation_rate) for individual in offsprings]
        
        # New Fitness
        new_fitness = []
        for individual in mutated_offspring_population:
            fitness = evaluate_fitness(individual, weights)
            new_fitness.append(fitness)

        # Replace the population with the new generation
        population = selection(mutated_offspring_population, new_fitness, population_size)
        
        best_index = new_fitness.index(max(new_fitness))
        if new_fitness[best_index] > best_fitness:
            if check_constraints(population[best_index], chromosome_length):
                best_fitness = new_fitness[best_index]
                best_solution = population[best_index]
        # Termination condition
        if best_fitness == sum([weights[i][i] for i in range(chromosome_length)]):
            print(best_solution)
            return 1 
    return -1

# Test the complete genetic algorithm
population_size = 16
N = 5
generations = 1000
mutation_rate = 0.3
weights = []
for _ in range(N):
    weights.append(random.choices(range(1, N + 1), k=N))

if genetic_algorithm(population_size, N, generations, mutation_rate, weights) == -1:
    print("no solution")

[1, 3, 5, 2, 4]
